In [1]:
import torch
import pandas as pd
import matplotlib.pyplot as plt
from torchvision.transforms import Normalize
from collections import namedtuple
import torch.nn as nn

#importing Technical Analysis tools
from ta import trend
from ta import volatility
from ta import momentum

import utils
from utils import return_fun

In [2]:
reuters=pd.read_csv("data/reut_data.csv",sep='|')
sx5e_data=pd.read_csv("data\SX5E.csv")

In [3]:
sx5e_data = sx5e_data.sort_index(axis=0 ,ascending=False)
sx5e_data.reset_index(drop=True,inplace=True)
sx5e_data['Date']=pd.to_datetime(sx5e_data.Date)

sx5e_data.dropna(inplace=True)


adi=trend.ADXIndicator(close=sx5e_data[' Close'],high=sx5e_data[' High'],low=sx5e_data[' Low'])
m_acd=trend.MACD(close=sx5e_data[' Close'])

r_si=momentum.RSIIndicator(close=sx5e_data[' Close'])
stochastic_oscilator=momentum.StochasticOscillator(close=sx5e_data[' Close'],high=sx5e_data[' High'],low=sx5e_data[' Low'])
williamsr=momentum.WilliamsRIndicator(high=sx5e_data[' High'],low=sx5e_data[' Low'],close=sx5e_data[' Close'])

bollinger=volatility.BollingerBands(close=sx5e_data[' Close'])
atr=volatility.AverageTrueRange(high=sx5e_data[' High'],low=sx5e_data[' Low'],close=sx5e_data[' Close'])

exponential_moving_avarge9=trend.EMAIndicator(close=sx5e_data[' Close'],window=9)
exponential_moving_avarge18=trend.EMAIndicator(close=sx5e_data[' Close'],window=18)
exponential_moving_avarge30=trend.EMAIndicator(close=sx5e_data[' Close'],window=30)

sx5e_data['ADX']=adi.adx()
sx5e_data['MACD']=m_acd.macd()
sx5e_data['RSI']=r_si.rsi()
sx5e_data['Stoch Osc']=stochastic_oscilator.stoch()
sx5e_data['Williams R']=williamsr.williams_r()
sx5e_data['Bollinger High band']=bollinger.bollinger_hband()
sx5e_data['Bolinger Low Band']=bollinger.bollinger_lband()
sx5e_data['ATR']=atr.average_true_range()
sx5e_data['EMA9']=exponential_moving_avarge9.ema_indicator()
sx5e_data['EMA18']=exponential_moving_avarge18.ema_indicator()
sx5e_data['EMA30']=exponential_moving_avarge30.ema_indicator()
sx5e_data.dropna(inplace=True)
ret_fun=return_fun(sx5e_data,1,4)
sx_ret=sx5e_data.copy()
sx_ret['Ret']=ret_fun
sx_ret.drop(columns=['ADX','MACD','RSI','Stoch Osc','Williams R','Bollinger High band','Bolinger Low Band','ATR','EMA9','EMA18','EMA30',' Open',' High',' Low'],inplace=True)

c:\users\48694\appdata\local\programs\python\python39\lib\site-packages\ta\trend.py:768: RuntimeWarning: invalid value encountered in double_scalars
  dip[i] = 100 * (self._dip[i] / self._trs[i])
c:\users\48694\appdata\local\programs\python\python39\lib\site-packages\ta\trend.py:772: RuntimeWarning: invalid value encountered in double_scalars
  din[i] = 100 * (self._din[i] / self._trs[i])


In [4]:
reuters.drop(columns='column1',inplace=True)
reuters.drop_duplicates(subset ="Headline",
                     keep = 'first', inplace = True)
reuters.dropna(inplace=True)
reuters.reset_index(drop=True,inplace=True)

for i in range(len(reuters)):
    text=reuters['Headline'][i]
    text=text.replace("REFILE-UPDATE 2-","")
    text=text.replace("UPDATE 2-","")
    text=text.replace("UPDATE 1-","")
    text=text.replace("UPDATE ","")
    text=text.replace(",","")
    text=text.replace(".","")
    text=text.replace(";","")
    text=text.replace(":","")
    text=text.replace("-","")
    text=text.replace("European","")
    reuters['Headline'][i]=text

In [5]:
sx5e_data.drop(columns='Date',inplace=True)
sx5e_data.drop(sx5e_data.tail(1).index,inplace=True)
sx5e_tensor=torch.tensor(sx5e_data.values)

In [6]:
mean=torch.mean(sx5e_tensor,dim=0)
std=torch.std(sx5e_tensor,dim=0)
sx5e_norm=(sx5e_tensor-mean)/std
print(sx5e_norm)

tensor([[-0.9221, -0.9442, -1.0001,  ..., -0.9280, -0.8126, -0.7074],
        [-0.9531, -0.9766, -0.9926,  ..., -0.9452, -0.8345, -0.7281],
        [-0.9801, -0.9924, -0.9511,  ..., -0.9550, -0.8519, -0.7462],
        ...,
        [ 2.1637,  2.3829,  2.2003,  ...,  2.2373,  2.3354,  2.4011],
        [ 2.4110,  2.4069,  2.3607,  ...,  2.2597,  2.3400,  2.4022],
        [ 2.3454,  2.3174,  2.2942,  ...,  2.2670,  2.3384,  2.3997]],
       dtype=torch.float64)


In [7]:
ret_fun.remove(None)
tensor_ret=torch.tensor(ret_fun,dtype=torch.float32)
mean_ret=torch.mean(tensor_ret)
std_ret=torch.std(tensor_ret)

In [8]:
sx_ret.dropna(inplace=True)

In [9]:
print(sx5e_norm.shape)
print(sx_ret.shape)

torch.Size([3076, 15])
(3076, 3)


In [10]:

lang=utils.Lang('t')

for i in range(len(reuters)):
    sentence=reuters['Headline'][i]
    lang.addSentence(sentence)

In [11]:
data_set=[]
prediction_days=5

d=namedtuple('d',['headline','numeric'])
data=namedtuple('Data',['data','target'])
for i in range(len(reuters)):
    headline=utils.tensorFromSentence(lang,reuters['Headline'][i])
    row=sx_ret.loc[sx_ret['Date']== reuters['Date'][i]]
    idx=row.index
    if(idx.empty):
        continue
    if(idx-29-prediction_days <0):
        continue
    numeric=sx5e_norm[idx[0]-29-5 : idx[0]-29]
    ret=row['Ret']
    if(ret.empty):
        continue
    ret=torch.tensor(ret.values)
    r=(ret-mean_ret)/std_ret
    d1=d(headline,numeric)
    da=data(d1,r)
    data_set.append(da)
    
    

In [12]:
train_data=data_set[3232:]
val_data=data_set[1616:3232]
test_data=data_set[:1616]

print(len(train_data))
print(len(val_data))
print(len(test_data))

9224
1616
1616


In [13]:
data_set[0].data.numeric.shape

torch.Size([5, 15])

In [14]:
train_loader=torch.utils.data.DataLoader(train_data, batch_size=1,shuffle=True)
val_loader=torch.utils.data.DataLoader(val_data, batch_size=1)
test_loader=torch.utils.data.DataLoader(test_data, batch_size=1)

In [15]:
for i, (x, y) in enumerate(train_loader):
    print(x.numeric.shape)
    break

torch.Size([1, 5, 15])


In [16]:
from utils import TextAndNumeric
from utils import correct

In [17]:
text_and_numeric=TextAndNumeric(lang.n_words,16,16,15).cuda()

params = text_and_numeric.parameters()
# for name, param in lstm_numeric.named_parameters():
#     if param.requires_grad:
#         print(param.data) 
        
loss_fn=torch.nn.MSELoss()
optimizer = torch.optim.Adam(params, lr=0.0015) 
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.7)

epoch = 5


history_train_loss=[]
history_val_loss=[]
history_train_acc=[]
history_val_acc=[]

# main loop
for e in range(1,epoch+1):
    
    train_losses=[]
    train_acc=[]
    
    loss_buffer=[]
    train_correct=0
    
    text_and_numeric.train()
    
    for i, (x, y) in enumerate(train_loader):
        headline,numeric = x
        numeric=numeric.float()
        headline=headline.cuda()
        numeric=numeric.cuda()
        y = y.cuda()
        
        optimizer.zero_grad()
        
        output = text_and_numeric(headline,numeric)
        output=output.float()
        y=y.float()
        loss = loss_fn(output, y)     
        train_correct+=correct(output,y,std_ret,mean_ret)
        loss_buffer.append(loss.item())
        loss.backward()
        optimizer.step() 
        if(i%1000==1):
            print(f"Epoch: {e} Progress: {100 * i/len(train_loader):2.0f}% Loss: {torch.mean(torch.tensor(loss_buffer)):.3f}")
        
    
    scheduler.step()
    
    train_loss=torch.mean(torch.tensor(loss_buffer))
    history_train_loss.append(train_loss)
    train_acc=train_correct/len(train_loader)
    history_train_acc.append(train_acc)
    
    
    print()
    print(f"Epoch:  {e}  Train Loss:  {train_loss:.3f} Train Acc: {train_acc:.5f}")
    
    text_and_numeric.eval()
    with torch.no_grad():
        val_loss_buffer=[]
        val_ac=0
        cor=0
        for i, (x,y) in enumerate(val_loader):
            headline,numeric = x
            numeric=numeric.float()
            headline=headline.cuda()
            numeric=numeric.cuda()
            y=y.cuda()
            
            output=text_and_numeric(headline,numeric)
            output.float()
            y.float()
            val_loss_buffer.append(loss_fn(output,y).item())
            
            
            cor+=correct(output,y,std_ret,mean_ret)
            
        val_acc=cor/len(val_loader)
        val_loss=torch.mean(torch.tensor(val_loss_buffer))
        print(f"Val loss: {val_loss:.3f} Val Acc {val_acc:.5f}")
        print()
        
        history_val_loss.append(val_loss)
        history_val_acc.append(val_acc)

Epoch: 1 Progress:  0% Loss: 0.065
Epoch: 1 Progress: 11% Loss: 1.036
Epoch: 1 Progress: 22% Loss: 1.043
Epoch: 1 Progress: 33% Loss: 1.062
Epoch: 1 Progress: 43% Loss: 1.057
Epoch: 1 Progress: 54% Loss: 1.087
Epoch: 1 Progress: 65% Loss: 1.100
Epoch: 1 Progress: 76% Loss: 1.094
Epoch: 1 Progress: 87% Loss: 1.106
Epoch: 1 Progress: 98% Loss: 1.107

Epoch:  1  Train Loss:  1.107 Train Acc: 0.50455
Val loss: 0.891 Val Acc 0.49505

Epoch: 2 Progress:  0% Loss: 0.528
Epoch: 2 Progress: 11% Loss: 0.952
Epoch: 2 Progress: 22% Loss: 1.015
Epoch: 2 Progress: 33% Loss: 1.044
Epoch: 2 Progress: 43% Loss: 1.022
Epoch: 2 Progress: 54% Loss: 1.057
Epoch: 2 Progress: 65% Loss: 1.073
Epoch: 2 Progress: 76% Loss: 1.080
Epoch: 2 Progress: 87% Loss: 1.097
Epoch: 2 Progress: 98% Loss: 1.107

Epoch:  2  Train Loss:  1.103 Train Acc: 0.50141
Val loss: 0.870 Val Acc 0.48267

Epoch: 3 Progress:  0% Loss: 0.708
Epoch: 3 Progress: 11% Loss: 1.129
Epoch: 3 Progress: 22% Loss: 1.054
Epoch: 3 Progress: 33% Loss: 

In [18]:
torch.save(text_and_numeric,'models/text_and_numeric')